In [9]:
import geopandas as gpd
import os
import fiona
from shapely.geometry import Polygon, MultiPolygon, LineString, MultiLineString, MultiPoint, Point
from shapely.geometry.polygon import orient
from shapely.geometry import mapping
import json

geopackage_path = "input/gadm41_IND.gpkg"

layer_names = fiona.listlayers(geopackage_path)
output_dir = "output/geoparquet"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Allowed geometry types for geoshape
allowed_types = (Polygon, MultiPolygon, LineString, MultiLineString, MultiPoint)

# Helper for fixing polygons
def fix_geom(geom):
    if isinstance(geom, (Polygon, MultiPolygon)):
        if not geom.is_valid:
            geom = geom.buffer(0)
        # Orient exterior rings counterclockwise
        if isinstance(geom, Polygon):
            geom = orient(geom, sign=1.0)
        elif isinstance(geom, MultiPolygon):
            geom = MultiPolygon([orient(p, sign=1.0) for p in geom.geoms])
    return geom

# Helper for ensuring numeric coordinates in GeoJSON
def ensure_numeric_coords(geojson_geom):
    def convert_coords(coords):
        if isinstance(coords, (list, tuple)):
            return [convert_coords(c) for c in coords]
        try:
            return float(coords)
        except Exception:
            return coords
    if 'coordinates' in geojson_geom:
        geojson_geom['coordinates'] = convert_coords(geojson_geom['coordinates'])
    return geojson_geom

# Helper for geopoint string
def point_to_geopoint(geom):
    if geom is not None and isinstance(geom, Point):
        # WGS84: (lat, lon)
        return f"{geom.y},{geom.x}"
    return None

for layer_name in layer_names:
    gdf = gpd.read_file(geopackage_path, layer=layer_name)

    # Set CRS to WGS84 (EPSG:4326) if it’s not already set
    if gdf.crs is None or gdf.crs.to_string() != "EPSG:4326":
        gdf = gdf.to_crs("EPSG:4326")

    # Prepare columns for Foundry
    gdf["geoshape"] = None

    # For non-Point geometries, fix and export as GeoJSON string
    mask = gdf["geometry"].apply(lambda geom: isinstance(geom, allowed_types) and not isinstance(geom, Point))
    gdf.loc[mask, "geometry"] = gdf.loc[mask, "geometry"].apply(fix_geom)
    gdf.loc[mask, "geoshape"] = gdf.loc[mask, "geometry"].apply(lambda geom: json.dumps(ensure_numeric_coords(mapping(geom))) if geom is not None else None)

    # For Point geometries, geoshape should be None
    gdf.loc[~mask, "geoshape"] = None

    # Drop the geometry column
    output_file = os.path.join(output_dir, f"{layer_name}.parquet")
    gdf.drop(columns=["geometry"]).to_parquet(output_file, engine='pyarrow')

# Preparing Vector Data for Foundry Ontology

This notebook demonstrates how to convert vector data from a GeoPackage (.gpkg) into Parquet files suitable for Foundry's Ontology ingestion, following these requirements:

- **Polygons/Lines**: Exported as a `geoshape` property (GeoJSON Geometry string, not Feature/FeatureCollection, WGS84 CRS, valid, counterclockwise winding for polygons).
- **No Feature, FeatureCollection, or GeometryCollection**.
- **All geometries must comply with RFC 7946 (GeoJSON spec)**.

The notebook will output Parquet files with the correct schema for Foundry ingestion.

In [10]:
import pandas as pd
import pyarrow.parquet as pq

# List of ADM levels and their files
adm_levels = [0, 1, 2, 3]
dfs = []
for level in adm_levels:
    df = pd.read_parquet(f'output/geoparquet/ADM_ADM_{level}.parquet')
    df['adm_level'] = level
    dfs.append(df)

combined = pd.concat(dfs, ignore_index=True)
combined.to_parquet('output/geoparquet/india_adm_all_levels.parquet')

In [18]:
combined.tail()

,GID_0,COUNTRY,geoshape,adm_level,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,...,CC_2,HASC_2,GID_3,NAME_3,VARNAME_3,NL_NAME_3,TYPE_3,ENGTYPE_3,CC_3,HASC_3
3065,IND,India,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[88...",3,IND.36_1,West Bengal,NaN,NA,NaN,NaN,...,NaN,NaN,IND.36.19.1_1,Diamond Harbour,NA,NA,Taluk,Taluk,NA,NA
3066,IND,India,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[88...",3,IND.36_1,West Bengal,NaN,NA,NaN,NaN,...,NaN,NaN,IND.36.19.2_1,n.a. ( 1187),NA,NA,Taluk,Taluk,NA,NA
3067,IND,India,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[88...",3,IND.36_1,West Bengal,NaN,NA,NaN,NaN,...,NaN,NaN,IND.36.19.3_1,n.a. ( 1229),NA,NA,Taluk,Taluk,NA,NA
3068,IND,India,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[88...",3,IND.36_1,West Bengal,NaN,NA,NaN,NaN,...,NaN,NaN,IND.36.20.1_1,Islampur,NA,NA,Taluk,Taluk,NA,NA
3069,IND,India,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[88...",3,IND.36_1,West Bengal,NaN,NA,NaN,NaN,...,NaN,NaN,IND.36.20.2_1,Raiganj,NA,NA,Taluk,Taluk,NA,NA


In [17]:
combined.columns

Index(['GID_0', 'COUNTRY', 'geoshape', 'adm_level', 'GID_1', 'NAME_1',
       'VARNAME_1', 'NL_NAME_1', 'TYPE_1', 'ENGTYPE_1', 'CC_1', 'HASC_1',
       'ISO_1', 'GID_2', 'NAME_2', 'VARNAME_2', 'NL_NAME_2', 'TYPE_2',
       'ENGTYPE_2', 'CC_2', 'HASC_2', 'GID_3', 'NAME_3', 'VARNAME_3',
       'NL_NAME_3', 'TYPE_3', 'ENGTYPE_3', 'CC_3', 'HASC_3'],
      dtype='object')

In [ ]:
# Normalize the combined DataFrame: keep only adm_level, GID, NAME, geoshape
level_gid = {0: 'GID_0', 1: 'GID_1', 2: 'GID_2', 3: 'GID_3'}
level_name = {0: 'COUNTRY', 1: 'NAME_1', 2: 'NAME_2', 3: 'NAME_3'}

def extract_level_info(row):
    level = row['adm_level']
    gid = row.get(level_gid[level], None)
    name = row.get(level_name[level], None)
    return pd.Series({'adm_level': level, 'GID': gid, 'NAME': name, 'geoshape': row['geoshape']})

normalized = combined.apply(extract_level_info, axis=1)
normalized = normalized.drop_duplicates().reset_index(drop=True)

# Save the normalized DataFrame
normalized.to_parquet('output/geoparquet/india_adm_all_levels_normalized.parquet')
normalized.head()

In [2]:
import pandas as pd

# Define ADM levels and corresponding file paths
adm_levels = [0, 1, 2, 3]
file_template = 'output/geoparquet/ADM_ADM_{}.parquet'

# Read and label each ADM level, then concatenate
dfs = [
    pd.read_parquet(file_template.format(level)).assign(adm_level=level)
    for level in adm_levels
]
combined = pd.concat(dfs, ignore_index=True)

# Normalize: keep adm_level, GID, NAME, geoshape, and extra fields for each level
level_gid = {0: 'GID_0', 1: 'GID_1', 2: 'GID_2', 3: 'GID_3'}
level_name = {0: 'COUNTRY', 1: 'NAME_1', 2: 'NAME_2', 3: 'NAME_3'}
level_fields = {
    1: ['VARNAME_1', 'NL_NAME_1', 'TYPE_1', 'ENGTYPE_1', 'CC_1', 'HASC_1', 'ISO_1'],
    2: ['VARNAME_2', 'NL_NAME_2', 'TYPE_2', 'ENGTYPE_2', 'CC_2', 'HASC_2'],
    3: ['VARNAME_3', 'NL_NAME_3', 'TYPE_3', 'ENGTYPE_3', 'CC_3', 'HASC_3'],
}

def extract_level_info(row):
    level = row['adm_level']
    data = {
        'adm_level': level,
        'GID': row.get(level_gid.get(level), None),
        'NAME': row.get(level_name.get(level), None),
        'geoshape': row.get('geoshape', None)
    }
    # Add extra fields for each level if present
    for l in range(1, 4):
        if level == l:
            for field in level_fields[l]:
                data[field] = row.get(field, None)
    return pd.Series(data)

normalized = combined.apply(extract_level_info, axis=1).drop_duplicates().reset_index(drop=True)

# Save outputs
combined.to_parquet('output/geoparquet/india_adm_all_levels.parquet')
normalized.to_parquet('output/geoparquet/india_geo.parquet')
normalized.head()

,CC_1,CC_2,CC_3,ENGTYPE_1,ENGTYPE_2,ENGTYPE_3,GID,HASC_1,HASC_2,HASC_3,...,NL_NAME_2,NL_NAME_3,TYPE_1,TYPE_2,TYPE_3,VARNAME_1,VARNAME_2,VARNAME_3,adm_level,geoshape
0,NaN,NaN,NaN,NaN,NaN,NaN,IND,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[76..."
1,NaN,NaN,NaN,NaN,NaN,NaN,Z01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[75..."
2,NaN,NaN,NaN,NaN,NaN,NaN,Z04,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[78..."
3,NaN,NaN,NaN,NaN,NaN,NaN,Z05,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[80..."
4,NaN,NaN,NaN,NaN,NaN,NaN,Z07,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[94..."


In [20]:
normalized.tail()

,adm_level,GID,NAME,geoshape
3065,3,IND.36.19.1_1,Diamond Harbour,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[88..."
3066,3,IND.36.19.2_1,n.a. ( 1187),"{""type"": ""MultiPolygon"", ""coordinates"": [[[[88..."
3067,3,IND.36.19.3_1,n.a. ( 1229),"{""type"": ""MultiPolygon"", ""coordinates"": [[[[88..."
3068,3,IND.36.20.1_1,Islampur,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[88..."
3069,3,IND.36.20.2_1,Raiganj,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[88..."


In [17]:
import pandas as pd

# Define ADM levels and corresponding file paths
adm_levels = [0, 1, 2, 3]
file_template = 'output/geoparquet/ADM_ADM_{}.parquet'

# Read and label each ADM level, then concatenate
dfs = [
    pd.read_parquet(file_template.format(level)).assign(adm_level=level)
    for level in adm_levels
]
combined = pd.concat(dfs, ignore_index=True)

# Map for extracting the correct field for each level
level_gid = {0: 'GID_0', 1: 'GID_1', 2: 'GID_2', 3: 'GID_3'}
level_name = {0: 'COUNTRY', 1: 'NAME_1', 2: 'NAME_2', 3: 'NAME_3'}
level_varname = {1: 'VARNAME_1', 2: 'VARNAME_2', 3: 'VARNAME_3'}
level_nlname = {1: 'NL_NAME_1', 2: 'NL_NAME_2', 3: 'NL_NAME_3'}
level_type = {1: 'TYPE_1', 2: 'TYPE_2', 3: 'TYPE_3'}
level_engtype = {1: 'ENGTYPE_1', 2: 'ENGTYPE_2', 3: 'ENGTYPE_3'}
level_cc = {1: 'CC_1', 2: 'CC_2', 3: 'CC_3'}
level_hasc = {1: 'HASC_1', 2: 'HASC_2', 3: 'HASC_3'}
level_iso = {1: 'ISO_1'}

def extract_level_info(row):
    level = row['adm_level']
    data = {
        'adm_level': level,
        'GID': row.get(level_gid.get(level)),
        'NAME': row.get(level_name.get(level)),
        'geoshape': row.get('geoshape')
    }
    # Add only one field for each type, if present for this level
    data['VARNAME'] = row.get(level_varname.get(level)) if level in level_varname else None
    data['NL_NAME'] = row.get(level_nlname.get(level)) if level in level_nlname else None
    data['TYPE'] = row.get(level_type.get(level)) if level in level_type else None
    data['ENGTYPE'] = row.get(level_engtype.get(level)) if level in level_engtype else None
    data['CC'] = row.get(level_cc.get(level)) if level in level_cc else None
    data['HASC'] = row.get(level_hasc.get(level)) if level in level_hasc else None
    data['ISO'] = row.get(level_iso.get(level)) if level in level_iso else None
    return pd.Series(data)

normalized = combined.apply(extract_level_info, axis=1).drop_duplicates().reset_index(drop=True)
normalized.drop(columns=['CC','NL_NAME', 'ENGTYPE'], inplace=True)

# Save outputs
normalized.to_parquet('output/geoparquet/india_geo.parquet')
normalized.head()

,adm_level,GID,NAME,geoshape,VARNAME,TYPE,HASC,ISO
0,0,IND,India,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[76...",None,None,None,None
1,0,Z01,India,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[75...",None,None,None,None
2,0,Z04,India,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[78...",None,None,None,None
3,0,Z05,India,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[80...",None,None,None,None
4,0,Z07,India,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[94...",None,None,None,None


array([None, 'NA', 'IN-AP', 'IN-AR', 'IN-AS', 'IN-BR', 'IN-CH', 'IN-CT',
       'IN-DN', 'IN-GA', 'IN-GJ', 'IN-HR', 'IN-HP', 'IN-JH', 'IN-KA',
       'IN-KL', 'IN-LD', 'IN-MP', 'IN-MH', 'IN-MN', 'IN-ML', 'IN-MZ',
       'IN-NL', 'IN-OR', 'IN-PY', 'IN-PB', 'IN-RJ', 'IN-SK', 'IN-TN',
       'IN-TG', 'IN-TR', 'IN-UP', 'IN-UT', 'IN-WB'], dtype=object)